In [13]:
%reload_ext autoreload

# Compare the df from the raw word list that contains the dutch and english dictionaries to preprocessed

In [14]:
import pandas as pd
import numpy as np
import json
from dutchanalyzer.config import *
from dutchanalyzer.utils import *
from pathlib import Path
from dotenv import load_dotenv
from io import StringIO
import datetime
import re

In [ ]:
# File Paths
en_postp_file = KAIKKI_EN_POSTPROCESSED_JSON
today_date = str(datetime.date.today())
en_nl_postp_file = Path(RAW_KAIKKI_DIR, 'nl', 'kaikki_nl-postproccessed.jsonl')
en_raw_file = Path(RAW_KAIKKI_DIR, 'en', 'kaikki_en-raw-wiktextract-data.jsonl')
interim_save_path = Path(INTERIM_DATA_DIR, 'exploration', '2025-10-29')
old_combined_csv = Path(interim_save_path, 'all_en_nl_mul_entries_transfer.csv')
combined_csv_28_10_25 = Path(INTERIM_DATA_DIR, 'exploration', '28-10-2025_2')
nlwikt_nld_eng_wikt_28_10_25 = Path(INTERIM_DATA_DIR, 'preprocessing', 'nl-wikt', 'nlwikt_pp_nld-eng_concat_28-10.csv')
enwikt_nld_eng_wikt_28_10_25 = Path(INTERIM_DATA_DIR, 'preprocessing', 'en-wikt', 'enwikt_pp_nl-en_concat_28-10.csv')

## Initial Processing

In [ ]:
raw_df = pd.read_csv(old_combined_csv, index_col=0)
raw_df = raw_df[raw_df['lang_code']  != 'mul']

C:\Users\elise\AppData\Local\Temp\ipykernel_3528\915289587.py:1: DtypeWarning: Columns (9,12,15,26,27,28,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(old_combined_csv)


In [16]:
print(raw_df.shape)
raw_word_set = raw_df['word'].unique()
print(len(raw_word_set))

(1558932, 36)
1446918


In [ ]:
en_wiki = pd.read_csv(enwikt_nld_eng_wikt_28_10_25, index_col=0)
nl_wiki = pd.read_csv(nlwikt_nld_eng_wikt_28_10_25, index_col=0)

C:\Users\elise\AppData\Local\Temp\ipykernel_3528\2999059573.py:1: DtypeWarning: Columns (4,13,16,17,18,20,21,22,23,25,26,30,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  en_wiki = pd.read_csv(enwikt_nld_eng_wikt_28_10_25, index_col=0)
C:\Users\elise\AppData\Local\Temp\ipykernel_3528\2999059573.py:2: DtypeWarning: Columns (22,23,24,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  nl_wiki = pd.read_csv(nlwikt_nld_eng_wikt_28_10_25)


In [11]:
nl_wiki['origin'] = ['pp_nl']*len(nl_wiki)
en_wiki['origin'] = ['pp_en']* len(en_wiki)

In [12]:
pp_concat_df = pd.concat([en_wiki, nl_wiki], ignore_index=True)

In [13]:
display(pp_concat_df)

,word,pos,lang_code,abbreviations,antonyms,categories,coordinate_terms,derived,descendants,etymology_number,...,origin,pos_title,etymology_texts,notes,tags,proverbs,metonyms,anagrams,homophones,paronyms
0,dictionary,verb,en,NaN,NaN,NaN,NaN,"[{'word': 'undictionaried', '_dis1': '0 0 0'}]",NaN,NaN,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,free,adv,en,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,free,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,encyclopedia,noun,en,NaN,NaN,NaN,NaN,"[{'word': 'encyclopedial', '_dis1': '0 0 0'}, ...",NaN,NaN,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,portmanteau,adj,en,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096875,decahedron,noun,en,NaN,NaN,"[{'name': 'Woorden in het Engels', 'kind': 'ot...",NaN,NaN,NaN,NaN,...,pp_nl,Zelfstandig naamwoord,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1096876,octahedron,noun,en,NaN,NaN,"[{'name': 'Woorden in het Engels', 'kind': 'ot...",NaN,NaN,NaN,NaN,...,pp_nl,Zelfstandig naamwoord,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1096877,deceit,noun,en,NaN,NaN,"[{'name': 'Woorden in het Engels', 'kind': 'ot...",NaN,NaN,NaN,NaN,...,pp_nl,Zelfstandig naamwoord,['Via Middelengels deceyte van Oudfrans deceit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1096878,stray,verb,en,NaN,NaN,"[{'name': 'Werkwoord in het Engels', 'kind': '...",NaN,NaN,NaN,NaN,...,pp_nl,Werkwoord,['Via Middelengels stray/strey van Angelsaksis...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
raw_df = raw_df.drop(columns=['Unnamed: 0'])

In [19]:
raw_df['origin'] = ['raw_cb']*len(raw_df)

In [20]:
display(raw_df.head())

,word,pos,lang_code,lang,senses,head_templates,categories,forms,form_of,derived,...,troponyms,coordinate_terms,abbreviations,instances,redirects,info_templates,original_title,wikidata,wikipedia,origin
0,dictionary,noun,en,English,[{'examples': [{'text': 'If you want to know t...,"[{'name': 'en-noun', 'args': {}, 'expansion': ...",['American Sign Language terms in nonstandard ...,"[{'form': 'dictionaries', 'tags': ['plural']},...",NaN,"[{'word': 'antidictionary'}, {'word': 'dicktio...",...,NaN,"[{'word': 'anagram dictionary', 'source': 'The...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb
1,dictionary,verb,en,English,"[{'categories': ['English transitive verbs'], ...","[{'name': 'en-verb', 'args': {}, 'expansion': ...",['American Sign Language terms in nonstandard ...,"[{'form': 'dictionaries', 'tags': ['present', ...",NaN,[{'word': 'undictionaried'}],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb
2,free,adj,en,English,[{'examples': [{'text': 'He was given free rei...,"[{'name': 'en-adj', 'args': {'1': 'er', '2': '...","['English adjectives', 'English adverbs', 'Eng...","[{'form': 'freer', 'tags': ['comparative']}, {...",NaN,"[{'word': 'break free'}, {'word': 'cofree'}, {...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb
3,free,adv,en,English,[{'examples': [{'text': 'I got this bike free....,"[{'name': 'en-adv', 'args': {}, 'expansion': '...","['English adjectives', 'English adverbs', 'Eng...","[{'form': 'more free', 'tags': ['comparative']...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb
4,free,verb,en,English,"[{'examples': [{'text': 'Pro.[…]Spirit, fine ſ...","[{'name': 'en-verb', 'args': {}, 'expansion': ...","['English countable nouns', 'English entries w...","[{'form': 'frees', 'tags': ['present', 'singul...",NaN,"[{'word': 'befree'}, {'word': 'free up'}]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb


In [21]:
new_raw = raw_df.sort_values('word')

In [22]:
pp_concat_df = pp_concat_df.sort_values('word')

In [23]:
display(pp_concat_df)

,word,pos,lang_code,abbreviations,antonyms,categories,coordinate_terms,derived,descendants,etymology_number,...,origin,pos_title,etymology_texts,notes,tags,proverbs,metonyms,anagrams,homophones,paronyms
6711,!,symbol,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272076,!Kung,name,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
783075,!Xóõ,name,nl,NaN,NaN,"[{'name': 'Eigennaam in het Nederlands', 'kind...",NaN,NaN,NaN,NaN,...,pp_nl,Eigennaam,NaN,NaN,['neuter'],NaN,NaN,NaN,NaN,NaN
1088377,!Xóõ,noun,en,NaN,NaN,"[{'name': ""Ta'atalen"", 'kind': 'other', 'paren...",NaN,NaN,NaN,NaN,...,pp_nl,Zelfstandig naamwoord,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
661616,"""R"" slur",noun,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2521,NaN,name,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5565,NaN,adj,en,NaN,"[{'word': 'antinull'}, {'word': 'non-null'}]",NaN,NaN,"[{'word': '/dev/null', '_dis1': '0 0 0 0 0 0 0...",NaN,NaN,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7855,NaN,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16168,NaN,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,pp_en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def check_has_invalid_chars(text):
    allowed_chars = (
        "a-zA-Z"                   # Standard letters
        "áéíóúÁÉÍÓÚ"               # Acute accents
        "èàòùÈÀÒÙ"                 # Grave accents
        "ëïöüËÏÖÜ"                 # Diaeresis
        "'\\- "                     # Apostrophe, hyphen, space
        "0-9"                       # Digits
        "\\.,;:"                  # Common punctuation
    )
    try: 
        text = str(text)
        text = text.strip()
        invalid_char_pattern = re.compile(f"[^{allowed_chars}]")
        ans = bool(invalid_char_pattern.search(text))
        return bool(invalid_char_pattern.search(text))
    except: 
        return True

In [25]:
invalid = []
for i, x in pp_concat_df.iterrows():
    w = x['word']
    val = check_has_invalid_chars(w)
    invalid.append(val)

! is True


In [27]:
pp_concat_df['invalid'] = invalid

In [28]:
pp_concat_df

,word,pos,lang_code,abbreviations,antonyms,categories,coordinate_terms,derived,descendants,etymology_number,...,pos_title,etymology_texts,notes,tags,proverbs,metonyms,anagrams,homophones,paronyms,invalid
6711,!,symbol,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
272076,!Kung,name,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
783075,!Xóõ,name,nl,NaN,NaN,"[{'name': 'Eigennaam in het Nederlands', 'kind...",NaN,NaN,NaN,NaN,...,Eigennaam,NaN,NaN,['neuter'],NaN,NaN,NaN,NaN,NaN,True
1088377,!Xóõ,noun,en,NaN,NaN,"[{'name': ""Ta'atalen"", 'kind': 'other', 'paren...",NaN,NaN,NaN,NaN,...,Zelfstandig naamwoord,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
661616,"""R"" slur",noun,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2521,NaN,name,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5565,NaN,adj,en,NaN,"[{'word': 'antinull'}, {'word': 'non-null'}]",NaN,NaN,"[{'word': '/dev/null', '_dis1': '0 0 0 0 0 0 0...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
7855,NaN,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
16168,NaN,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [29]:
pp_concat_df = pp_concat_df[pp_concat_df['invalid'] != True]

In [32]:
pp_concat_df = pp_concat_df[~pp_concat_df['word'].isna()]

In [33]:
pp_concat_df

,word,pos,lang_code,abbreviations,antonyms,categories,coordinate_terms,derived,descendants,etymology_number,...,pos_title,etymology_texts,notes,tags,proverbs,metonyms,anagrams,homophones,paronyms,invalid
23150,',symbol,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
273773,'00s,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
344289,'30s,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
344288,'40s,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
594828,'50s progression,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031947,überseksuelers,adj,nl,NaN,NaN,"[{'name': 'Ontbrekend geluid', 'kind': 'other'...",NaN,NaN,NaN,NaN,...,Bijvoeglijk naamwoord,['überseksueler met de uitgang -s'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
256590,übersexual,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
584544,ümlauts,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
581146,üntermensch,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [35]:
cols = pp_concat_df.columns
cols = cols.sort_values()

In [ ]:
cols = ['word', 'pos', 'lang_code', 'senses', 'abbreviations', 'anagrams', 'antonyms', 'categories',
       'coordinate_terms', 'derived', 'descendants', 'etymology_number',
       'etymology_templates', 'etymology_text', 'etymology_texts', 'form_of',
       'forms', 'head_templates', 'holonyms', 'homophones', 'hypernyms',
       'hyphenation', 'hyphenations', 'hyponyms', 'inflection_templates',
       'info_templates', 'instances', 'invalid', 'lang', 
       'meronyms', 'metonyms', 'notes', 'origin', 'original_title', 'paronyms',
        'pos_title', 'proverbs', 'related',  'sounds', 'source',
       'synonyms', 'tags', 'translations', 'troponyms', 'wikidata',
       'wikipedia']

In [ ]:
pp_concat_df = pp_concat_df.loc[:, cols]

In [ ]:
pp_concat_df.to_csv(Path(interim_save_path, 'pp_concat_df_25-10-29.csv'))

### Raw DF Cleaning

In [42]:
invalid = []
for i, x in raw_df.iterrows():
    w = x['word']
    val = check_has_invalid_chars(w)
    invalid.append(val)

! is True


In [44]:
raw_df['invalid'] = invalid

In [46]:
raw_df = raw_df.sort_values('word')

In [ ]:
raw_df = raw_df[raw_df['invalid'] != True]

In [ ]:
raw_df = raw_df[~raw_df['word'].isna()]

In [ ]:
cols = raw_df.columns
cols = cols.sort_values()
start_cols = ['word', 'pos', 'lang_code', 'senses']
cols = start_cols + [i for i in cols if i not in start_cols]
raw_df = raw_df.loc[:, cols]

,word,pos,lang_code,lang,senses,head_templates,categories,forms,form_of,derived,...,coordinate_terms,abbreviations,instances,redirects,info_templates,original_title,wikidata,wikipedia,origin,invalid
452461,,punct,en,English,"[{'links': [['Internet', 'Internet'], ['vaporw...","[{'name': 'head', 'args': {'1': 'en', '2': 'pu...",NaN,"[{'form': '] [', 'tags': ['canonical']}]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Unsupported titles/Space,NaN,NaN,raw_cb,False
73735,',particle,en,English,"[{'links': [[""-'"", ""-'#English""]], 'glosses': ...","[{'name': 'head', 'args': {'1': 'en', '2': 'pa...",['English entries with incorrect language head...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb,False
73734,',symbol,en,English,"[{'links': [['’', '’#English']], 'glosses': ['...","[{'name': 'head', 'args': {'1': 'en', '2': 'sy...",['English entries with incorrect language head...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb,False
660135,'00s,noun,en,English,[{'examples': [{'text': 'Since first tossing i...,"[{'name': 'en-noun', 'args': {'1': 'p'}, 'expa...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb,False
826193,'10s,noun,en,English,"[{'links': [['1910s', '1910s']], 'glosses': ['...","[{'name': 'en-noun', 'args': {'1': 'p'}, 'expa...",['English entries with incorrect language head...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623612,übersexuals,noun,en,English,"[{'links': [['übersexual', 'übersexual#English...","[{'name': 'head', 'args': {'1': 'en', '2': 'no...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb,False
1367585,ümlaut,noun,en,English,[{'examples': [{'text': 'The Traveler said he ...,"[{'name': 'en-noun', 'args': {}, 'expansion': ...",NaN,"[{'form': 'ümlauts', 'tags': ['plural']}]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb,False
1367586,ümlauts,noun,en,English,"[{'links': [['ümlaut', 'ümlaut#English']], 'gl...","[{'name': 'head', 'args': {'1': 'en', '2': 'no...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb,False
1360162,üntermensch,noun,en,English,"[{'categories': ['English countable nouns', 'E...","[{'name': 'en-noun', 'args': {'1': 'üntermensc...",NaN,"[{'form': 'üntermenschen', 'tags': ['plural']}]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raw_cb,False


In [61]:
invalid = []
for i, r in raw_df.iterrows():
    w = r['word']
    inv = r['invalid']
    if inv == True:
        invalid.append(True)
    elif w[0].isnumeric():
        invalid.append(True)
    elif len(w) > 1:
        if w[1].isnumeric():
            invalid.append(True)
        else:
            invalid.append(False)
    elif w == ' ':
        invalid.append(True)
    else:
        invalid.append(False)
raw_df['invalid'] = invalid

In [ ]:
raw_df.to_csv(Path(interim_save_path, 'raw_df_25-10-29.csv'), index=0)

## Setup Unique Dicts

In [24]:
current_save_path = Path(INTERIM_DATA_DIR, 'exploration', '2025-10-30')
last_save_path = Path(INTERIM_DATA_DIR, 'exploration', '2025-10-29')
last_raw_df_path = Path(last_save_path, 'filtered_raw_df_25-10-29.csv')
last_pp_concat_path = Path(last_save_path, 'pp_concat_df_25-10-29.csv')

In [22]:
raw_df = pd.read_csv(last_raw_df_path)

C:\Users\elise\AppData\Local\Temp\ipykernel_11644\763911384.py:1: DtypeWarning: Columns (8,11,14,25,27,29,30,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(last_raw_df_path)


In [25]:
pp_concat_df = pd.read_csv(last_pp_concat_path)

C:\Users\elise\AppData\Local\Temp\ipykernel_11644\4187455787.py:1: DtypeWarning: Columns (5,6,16,19,20,22,26,27,30,31,35,40,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  pp_concat_df = pd.read_csv(last_pp_concat_path)


In [35]:
pp_concat_df = pp_concat_df.drop(columns=['Unnamed: 0'])

In [ ]:
cols = raw_df.columns
cols = cols.sort_values()
start_cols = ['word', 'pos', 'lang_code', 'senses']
cols = start_cols + [i for i in cols if i not in start_cols]
cols.remove('invalid')
cols = cols + ['invalid']
raw_df = raw_df.loc[:, cols]

In [65]:
cols = cols + ['invalid']
raw_df = raw_df.loc[:, cols]

In [72]:
raw_df.head()

,word,pos,lang_code,senses,abbreviations,alt_of,antonyms,categories,coordinate_terms,derived,...,original_title,redirects,related,sounds,synonyms,translations,troponyms,wikidata,wikipedia,invalid
1,',particle,en,"[{'links': [[""-'"", ""-'#English""]], 'glosses': ...",NaN,NaN,NaN,['English entries with incorrect language head...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,',symbol,en,"[{'links': [['’', '’#English']], 'glosses': ['...",NaN,NaN,NaN,['English entries with incorrect language head...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
17,'Are'are,name,en,"[{'links': [['Oceanic', 'Oceanic']], 'glosses'...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"[{'word': ""Wiktionary’s coverage of 'Are'are t...",NaN,NaN,NaN,NaN,NaN,NaN,False
18,'Arries,noun,en,"[{'links': [[""'Arry"", ""'Arry#English""]], 'glos...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
19,'Arriet,noun,en,"[{'categories': ['English colloquialisms', 'En...",NaN,NaN,NaN,NaN,"[{'word': ""'Arry""}]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [91]:
print(raw_df.shape)
raw_word_set = raw_df['word'].unique()
print(len(raw_word_set))

(1547243, 38)
1435998


In [70]:
raw_df = raw_df[raw_df['invalid'] != True]

In [71]:
raw_df.to_csv(Path(current_save_path, 'raw_df_added_processing_25-30-10.csv'))

In [82]:
pp_concat_df.columns

Index(['word', 'pos', 'lang_code', 'senses', 'abbreviations', 'anagrams',
       'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants',
       'etymology_number', 'etymology_templates', 'etymology_text',
       'etymology_texts', 'form_of', 'forms', 'head_templates', 'holonyms',
       'homophones', 'hypernyms', 'hyphenation', 'hyphenations', 'hyponyms',
       'inflection_templates', 'info_templates', 'instances', 'invalid',
       'lang', 'meronyms', 'metonyms', 'notes', 'origin', 'original_title',
       'paronyms', 'pos_title', 'proverbs', 'related', 'sounds', 'source',
       'synonyms', 'tags', 'translations', 'troponyms', 'wikidata',
       'wikipedia'],
      dtype='object')

In [83]:
cols = pp_concat_df.columns
cols = cols.sort_values()
start_cols = ['word', 'pos', 'lang_code', 'senses']
cols = start_cols + [i for i in cols if i not in start_cols]
cols.remove('invalid')
cols = cols + ['invalid']
pp_concat_df = pp_concat_df.loc[:, cols]

In [85]:
invalid = []
for i, r in pp_concat_df.iterrows():
    w = r['word']
    inv = r['invalid']
    if inv == True:
        invalid.append(True)
    elif w[0].isnumeric():
        invalid.append(True)
    elif len(w) > 1:
        if w[1].isnumeric():
            invalid.append(True)
        else:
            invalid.append(False)
    elif w == ' ':
        invalid.append(True)
    else:
        invalid.append(False)
pp_concat_df['invalid'] = invalid

In [87]:
pp_concat_df = pp_concat_df[pp_concat_df['invalid'] != True]
pp_concat_df.to_csv(Path(current_save_path, 'pp_concat_df_25-10-30.csv'))

In [88]:
print(pp_concat_df.shape)
pp_concat_df.head()

(1089795, 46)


,word,pos,lang_code,senses,abbreviations,anagrams,antonyms,categories,coordinate_terms,derived,...,related,sounds,source,synonyms,tags,translations,troponyms,wikidata,wikipedia,invalid
0,',symbol,en,"[{'links': [['’', '’#English']], 'glosses': ['...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
6,'Arries,noun,en,"[{'links': [[""'Arry"", ""'Arry#English""]], 'glos...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
7,'Arriets,noun,en,"[{'links': [[""'Arriet"", ""'Arriet#English""]], '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
8,'Arry,noun,en,"[{'links': [['Cockney', 'Cockney']], 'raw_glos...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
9,'Arryish,adj,en,[{'examples': [{'text': 'He has a fair stock o...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [93]:
print(pp_concat_df.shape)
pp_word_set = pp_concat_df['word'].unique()
print(len(pp_word_set))

(1089795, 46)
1033086


In [94]:
raw_col = ['raw']*len(raw_word_set)
pp_col = ['pp']*len(pp_word_set)

In [95]:
raw_unique_df = pd.DataFrame({'word': raw_word_set, 'origin_df':raw_col})

In [96]:
pp_unique_df = pd.DataFrame({'word': pp_word_set, 'origin_df':pp_col})

In [97]:
all_words_df = pp_unique_df.merge(raw_unique_df, how='outer', left_on='word', right_on='word', indicator=True, suffixes=('_pp', '_r'))

In [98]:
display(all_words_df.shape)

(1679339, 4)

In [99]:
display(all_words_df)

,word,origin_df_pp,origin_df_r,_merge
0,',pp,raw,both
1,'Are'are,NaN,raw,right_only
2,'Arries,pp,raw,both
3,'Arriet,NaN,raw,right_only
4,'Arriets,pp,raw,both
...,...,...,...,...
1679334,übersexuals,NaN,raw,right_only
1679335,ümlaut,NaN,raw,right_only
1679336,ümlauts,pp,raw,both
1679337,üntermensch,pp,raw,both


In [100]:
non_pp = all_words_df[all_words_df['origin_df_pp'].isna() == True]

In [103]:
non_raw = all_words_df[all_words_df['origin_df_r'].isna() == True]

In [102]:
print(non_pp.shape)
non_pp[1000:1050]

(646253, 4)


,word,origin_df_pp,origin_df_r,_merge
2402,A-V,NaN,raw,right_only
2405,A-back,NaN,raw,right_only
2413,A-bomb maidens,NaN,raw,right_only
2415,A-bomber,NaN,raw,right_only
2416,A-bombers,NaN,raw,right_only
2417,A-bombing,NaN,raw,right_only
2420,A-choy,NaN,raw,right_only
2426,A-fib,NaN,raw,right_only
2429,A-flat major,NaN,raw,right_only
2432,A-frame,NaN,raw,right_only


In [105]:
print(non_raw.shape)
non_raw.head()

(243341, 4)


,word,origin_df_pp,origin_df_r,_merge
275,'matoes,pp,NaN,left_only
299,'nieuwe' tweezaadlobbigen,pp,NaN,left_only
394,'s Herenelderen,pp,NaN,left_only
403,'s mans,pp,NaN,left_only
411,'s weekends,pp,NaN,left_only


In [107]:
all_words_df['lower'] = all_words_df['word'].apply(lambda x: x.lower())

In [159]:
val_counts = all_words_df['lower'].value_counts()
words = val_counts.index.to_list()

In [ ]:
val_counts

In [141]:
words = [w.strip() for w in words]
words.sort()

In [176]:
all_words_df

,word,origin_df_pp,origin_df_r,_merge,lower
0,',pp,raw,both,'
1,'Are'are,NaN,raw,right_only,'are'are
2,'Arries,pp,raw,both,'arries
3,'Arriet,NaN,raw,right_only,'arriet
4,'Arriets,pp,raw,both,'arriets
...,...,...,...,...,...
1679334,übersexuals,NaN,raw,right_only,übersexuals
1679335,ümlaut,NaN,raw,right_only,ümlaut
1679336,ümlauts,pp,raw,both,ümlauts
1679337,üntermensch,pp,raw,both,üntermensch


In [160]:
val_df = pd.DataFrame({'lower': words, 'val_counts': val_counts})

In [162]:
val_df.reset_index(drop=True, inplace=True)

In [ ]:
val_df

,lower,val_counts
0,chip,5
1,ads,5
2,goes,5
3,amber alert,5
4,amber alerts,5


In [168]:
lower = val_df['lower'].sort_values()

In [170]:
val_df = val_df.sort_values(by='lower')

In [172]:
val_df['word_id'] = [i for i in range(len(val_df))]

In [177]:
id_words_df = all_words_df.merge(val_df, how='outer', on='lower')

,word,origin_df_pp,origin_df_r,_merge,lower,val_counts,word_id
0,',pp,raw,both,',1,0
1,'a,pp,raw,both,'a,1,1
2,'a',NaN,raw,right_only,'a',1,2
3,'a'a,pp,raw,both,'a'a,1,3
4,'ad,NaN,raw,right_only,'ad,1,4
...,...,...,...,...,...,...,...
1679334,Üntermenschen,NaN,raw,right_only,üntermenschen,2,1644311
1679335,üntermenschen,pp,raw,both,üntermenschen,2,1644311
1679336,Ürümchi,NaN,raw,right_only,ürümchi,1,1644312
1679337,Ürümqi,NaN,raw,right_only,ürümqi,1,1644313


In [ ]:
all_words_df.to_csv(Path(current_save_path, 'all_words_in_combo_dictionaries_2025-10-30.csv'))

In [181]:
id_words_df.to_csv(Path(current_save_path, 'id_all_words_in_combo_dictionaries_2025-10-30.csv'))

In [180]:
display(all_words_df[100:150])

,word,origin_df_pp,origin_df_r,_merge,lower
100,'bot,pp,raw,both,'bot
101,'bots,pp,raw,both,'bots
102,'bout,pp,raw,both,'bout
103,'bout ship,pp,raw,both,'bout ship
104,'bout shipped,pp,raw,both,'bout shipped
105,'bout shipping,NaN,raw,right_only,'bout shipping
106,'bout ships,pp,raw,both,'bout ships
107,'bouta,NaN,raw,right_only,'bouta
108,'boutcha,NaN,raw,right_only,'boutcha
109,'boutchu,NaN,raw,right_only,'boutchu


In [182]:
word_ids = id_words_df.loc[:, ['word', 'word_id']]

In [184]:
raw_df_idd = raw_df.merge(word_ids, on='word', how='left')

In [186]:
raw_df_idd.to_csv(Path(current_save_path, 'raw_df_word_id_25-30-10.csv'))

In [187]:
pp_concat_df_idd = pp_concat_df.merge(word_ids, on='word', how='left')

In [188]:
pp_concat_df_idd.to_csv(Path(current_save_path, 'pp_df_word_id_25-30-10.csv'))

## Final Merging of both dicts to get entry IDs

In [190]:
raw_df = raw_df_idd

In [ ]:
pp_concat_df = pp_concat_df_idd

Index(['word', 'pos', 'lang_code', 'senses', 'abbreviations', 'anagrams',
       'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants',
       'etymology_number', 'etymology_templates', 'etymology_text',
       'etymology_texts', 'form_of', 'forms', 'head_templates', 'holonyms',
       'homophones', 'hypernyms', 'hyphenation', 'hyphenations', 'hyponyms',
       'inflection_templates', 'info_templates', 'instances', 'lang',
       'meronyms', 'metonyms', 'notes', 'origin', 'original_title', 'paronyms',
       'pos_title', 'proverbs', 'related', 'sounds', 'source', 'synonyms',
       'tags', 'translations', 'troponyms', 'wikidata', 'wikipedia',
       'invalid'],
      dtype='object')

In [ ]:
pp_cols = pp_concat_df.columns
r_cols = raw_df.columns

In [197]:
col_match = {}
all_cols = pp_cols.to_list() + r_cols.to_list()
all_cols = set(all_cols)
for c in all_cols:
    in_pp = c in pp_cols
    in_r = c in r_cols
    if in_pp and in_r:
        col_match[c] = 'both'
    elif in_pp:
        col_match[c] = 'pp'
    elif in_r:
        col_match[c] = 'raw'

In [215]:
pp_concat_df['lower'] = pp_concat_df['word'].apply(lambda x: x.lower())

In [220]:
pp_concat_df = pp_concat_df.merge(word_ids, on='word', how='left')

In [224]:
raw_df.rename(inplace=True, columns={'etymology_text':'etymology_texts'})

In [221]:
pp_concat_df.columns

Index(['word', 'pos', 'lang_code', 'senses', 'abbreviations', 'anagrams',
       'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants',
       'etymology_number', 'etymology_templates', 'etymology_text',
       'etymology_texts', 'form_of', 'forms', 'head_templates', 'holonyms',
       'homophones', 'hypernyms', 'hyphenation', 'hyphenations', 'hyponyms',
       'inflection_templates', 'info_templates', 'instances', 'lang',
       'meronyms', 'metonyms', 'notes', 'origin', 'original_title', 'paronyms',
       'pos_title', 'proverbs', 'related', 'sounds', 'source', 'synonyms',
       'tags', 'translations', 'troponyms', 'wikidata', 'wikipedia', 'invalid',
       'lower', 'word_id'],
      dtype='object')

Index(['word', 'pos', 'lang_code', 'senses', 'abbreviations', 'alt_of',
       'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants',
       'etymology_number', 'etymology_templates', 'etymology_texts', 'form_of',
       'forms', 'head_templates', 'holonyms', 'hypernyms', 'hyphenation',
       'hyphenations', 'hyponyms', 'inflection_templates', 'info_templates',
       'instances', 'lang', 'meronyms', 'origin', 'original_title',
       'redirects', 'related', 'sounds', 'synonyms', 'translations',
       'troponyms', 'wikidata', 'wikipedia', 'invalid', 'word_id'],
      dtype='object')

In [ ]:
col_map = {}

In [204]:
diff = [item for item in col_match.items() if item[1] != 'both']

In [223]:
display(diff)

[('etymology_texts', 'pp'),
 ('proverbs', 'pp'),
 ('source', 'pp'),
 ('redirects', 'raw'),
 ('notes', 'pp'),
 ('word_id', 'raw'),
 ('metonyms', 'pp'),
 ('tags', 'pp'),
 ('paronyms', 'pp'),
 ('pos_title', 'pp'),
 ('alt_of', 'raw'),
 ('anagrams', 'pp'),
 ('homophones', 'pp')]

In [226]:
rough_merged_df = pd.concat([pp_concat_df, raw_df], ignore_index=True, join='outer')

In [228]:
rough_merged_df['lang_code'].value_counts()

lang_code
en    2121885
nl     515153
Name: count, dtype: int64

In [234]:
rough_merged_df.to_csv(Path(current_save_path, 'rough_merged_30-10-25.csv'))

,pos,lang_code,senses,abbreviations,anagrams,antonyms,categories,coordinate_terms,derived,descendants,...,tags,translations,troponyms,wikidata,wikipedia,invalid,lower,word_id,alt_of,redirects
word,,,,,,,,,,,,,,,,,,,,,
',True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,False,True,True
'Are'are,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,True
'Arries,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,True
'Arriet,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,True
'Arriets,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
übersexuals,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,True
ümlaut,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,True
ümlauts,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,True
